In [1]:
import pyterrier as pt
pt.init()
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex
from pyterrier_adaptive import GAR, CorpusGraph

/home/rathee/p-graph/p-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:


dataset = pt.get_dataset('irds:msmarco-passage')
retriever = PisaIndex.from_dataset('msmarco_passage').bm25()
scorer = pt.text.get_text(dataset, 'text') >> MonoT5ReRanker(verbose=False, batch_size=1)
graph = CorpusGraph.from_dataset('msmarco_passage', 'corpusgraph_bm25_k16').to_limit_k(16)
graph_2 = CorpusGraph.load("msmarco-passage.gbm25.1024").to_limit_k(16)

print(graph_2)
print("#"*100)
nbh = graph_2.neighbours(docid='0')



print("Neighbors", nbh, len(nbh))
print(len(graph_2.edges_data), graph_2.edges_data[100])
print(graph_2.weights_data)
print("#"*100)

graph_3 = graph_2.sparsify_randomly(5) 

nbh = graph_3.neighbours(docid='2180710')
print("Neighbors", nbh, len(nbh))
# print(len(graph_3.edges_data), graph_3.edges_data[100])
# print(graph_3.weights_data)




pipeline = retriever >> GAR(scorer, graph.sparsify_randomly(5)) >> pt.text.get_text(dataset, 'text')
search_results = pipeline.search('clustering hypothesis information retrieval')
print(search_results)



# qid                                        query    docno  rank       score  iteration                                               text
#   1  clustering hypothesis information retrieval  2180710     0   -0.017059          0  Cluster analysis or clustering is the task of ...
#   1  clustering hypothesis information retrieval  8430269     1   -0.166563          1  Clustering is the grouping of a particular set...
#   1  clustering hypothesis information retrieval  1091429     2   -0.208345          1  Clustering is a fundamental data analysis meth...
#   1  clustering hypothesis information retrieval  2180711     3   -0.341018          5  Cluster analysis or clustering is the task of ...
#   1  clustering hypothesis information retrieval  6031959     4   -0.367014          5  Cluster analysis or clustering is the task of ...
#  ..                                          ...      ...   ...         ...        ...                                                ...
#                iteration column indicates which GAR batch the document was scored in ^
#                even=initial retrieval   odd=corpus graph    -1=backfilled

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


NpTopKCorpusGraph('msmarco-passage.gbm25.1024', k=16)
####################################################################################################
Neighbors ['2942572' '0' '0' '3538204' '0' '2968764' '2036653' '2942574' '5759591'
 '1140006' '0' '0' '2422192' '3870084' '2036646' '4714605'] 16
8841823 [      0     103       0     102      99       0       0     101 3217998
 3217994 8766645       0 8766641 2483393 1857918 3217991]
[[ 41.6    0.     0.   ...  30.77  30.66  30.64]
 [ 55.22   0.    52.03 ...   0.    48.1   47.7 ]
 [ 47.2   40.47   0.   ...  37.44  37.12  36.94]
 ...
 [ 58.94  58.3   56.88 ...  54.53  54.44  53.4 ]
 [155.9   37.7   34.47 ...  30.95  30.61  30.05]
 [ 51.97  51.1   51.06 ...  44.38  44.3   44.12]]
####################################################################################################


KeyboardInterrupt: 

In [ ]:
from pyterrier.measures import *
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

print(dataset)
pt.Experiment(
    [retriever, retriever >> scorer, retriever >> GAR(scorer, graph)],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@1000],
    names=['bm25', 'bm25 >> monot5', 'bm25 >> GAR(monot5)']
)

In [5]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

print(dataset.get_topics())


        qid                                              query
0    156493                                   do goldfish grow
1   1110199                          what is wifi vs bluetooth
2   1063750              why did the us volunterilay enter ww1
3    130510                    definition declaratory judgment
4    489204                           right pelvic pain causes
5    573724         what are the social determinants of health
6    168216        does legionella pneumophila cause pneumonia
7   1133167                      how is the weather in jamaica
8    527433            types of dysarthria from cerebral palsy
9   1037798                                 who is robert gray
10   915593          what types of food can you cook sous vide
11   264014                     how long is life cycle of flea
12  1121402                    what can contour plowing reduce
13   962179                when was the salvation army founded
14  1117099                            what is a active

In [12]:
qrels_df  = dataset.get_qrels()
print(qrels_df.label)

0       0
1       0
2       0
3       0
4       0
       ..
9255    2
9256    2
9257    0
9258    0
9259    0
Name: label, Length: 9260, dtype: int64


In [12]:
import numpy as np

# Create a binary file with some data
data = np.arange(10)
with open('data.bin', 'wb') as f:
    data.tofile(f)

# Create a memory-mapped array to read data from the file
memmap_array = np.memmap('data.bin', dtype=np.int64, mode='r', shape=(10,))

# Access and print the memory-mapped array
print(memmap_array)


[0 1 2 3 4 5 6 7 8 9]


In [14]:
import numpy as np

# Load the data from the memory-mapped file
loaded_data = np.memmap('data.bin', dtype=np.float64, mode='r', shape=(5,))

# Print the loaded data
loaded_data_copy = loaded_data.copy()
loaded_data_copy[0] = 2

print(loaded_data_copy)

[2.0e+000 4.9e-324 9.9e-324 1.5e-323 2.0e-323]


In [11]:
loaded_data = np.memmap('data.bin', dtype=np.float64, mode='r+', shape=(5,))
loaded_data

memmap([2.0e+000, 4.9e-324, 9.9e-324, 1.5e-323, 2.0e-323])